# Creating training and testing data for segmentation


In [1]:
import napari
from macrohet import dataio, tile, visualise
import btrack
import os
from tqdm.auto import tqdm
from skimage.io import imread, imsave
import numpy as np

### Use cellpose to bootstrap segmentation maps

In [6]:
from cellpose import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

>>> GPU activated? 1


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [4]:
row = 3
column = 4

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
                             set_channel=1,
                             set_time = 1,
#                             input_transforms = [input_transforms]
                            ).compute().compute().astype(np.uint16)

### Load previous mask for checking

In [6]:
gt_fn = '/mnt/DATA/macrohet/upstream_development/ground_truth/semantic/gt_sem_r03c04f0*p*1-ch1sk2fk1fl1.tiff'

In [7]:
gt_masks = imread(gt_fn)

### Solve lag time by downscaling images

In [46]:
from skimage.transform import downscale_local_mean

def downscale_images_for_manual_annotation(image, labels, scale_factor):
    # Downscale the image
    downsampled_image = downscale_local_mean(image, (scale_factor, scale_factor))

    # Downscale the labels
    downsampled_labels = downscale_local_mean(labels.astype(float), (scale_factor, scale_factor))

    # Round the downsampled labels to the nearest integer
    downsampled_labels = np.round(downsampled_labels).astype(int)

    return downsampled_image, downsampled_labels

In [47]:
ds_images, ds_gt_masks = downscale_image_labels(images[0,0,...], gt_masks, scale_factor=4)

In [49]:
v = napari.Viewer()
v.add_image(ds_images, colormap='green', contrast_limits=[0,8000], blending = 'additive'
#             channel_axis=1,
#             name=["macrophage", "mtb"],
#             colormap=["green",  "magenta"],
#             contrast_limits=[[0,6000], [0,1000]], 
#             visible = True
           )
visualise.add_napari_grid_overlay(viewer = v, N_rows_cols=5)
v.add_labels(ds_gt_masks, color={1:'yellow'}, opacity=0.5, blending = 'additive')


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'ds_gt_masks' at 0x7fded5885f70>

In [81]:
viewer = v 

In [88]:
@viewer.bind_key("s", overwrite = True)
def save_masks(viewer):  # noqa: F811
    viewer.layers['ds_gt_masks'].save(gt_fn.replace('.tiff', '_updated.tiff'))
    print('Saved')

Saved


In [93]:
def upscale_labels_post_manual_annotation(labels, scale_factor):
    # Upscale the image using resize
    upscaled_labels = resize(labels, (labels.shape[0] * scale_factor, labels.shape[1] * scale_factor),
                            anti_aliasing=False, order=0, preserve_range=True)

    return upscaled_labels

In [109]:
v.add_labels(gt_mask_updated)

<Labels layer 'gt_mask_updated [1]' at 0x7fdede07e0d0>

In [107]:
gt_mask_updated = upscale_image(ds_gt_masks, 4).astype(np.uint8)

### Post editing save out

In [110]:
imsave(gt_fn, gt_mask_updated)

/tmp/ipykernel_24975/992389131.py:1: UserWarning: /mnt/DATA/macrohet/upstream_development/ground_truth/semantic/gt_sem_r03c04f0*p*1-ch1sk2fk1fl1_updated.tiff is a low contrast image
  imsave(gt_fn.replace('.tiff', '_updated.tiff'), gt_mask_updated)


In [111]:
np.save(gt_fn.replace('.tiff', '.npy'), gt_mask_updated)